# 模型

模型在深度学习中是最基本、最重要的概念。它代表了一个可用于训练（或推理）的数学整体。处理好的数据被输入到模型中，模型会给出预测的值以便使用，
也可以给出预测值和实际的标签值的差异（loss，或称为残差）以便于训练优化模型本身。

每个模型都有与之配套的[预处理过程](./数据的预处理.ipynb)，该过程可以将原始数据转换为模型可接受的张量型输入。

# ModelScope的模型

ModelScope提供了各模态的多种多样的模型，您可以登录[模型官网](www.modelscope.cn)来查看它们。
这些模型中很多模型可以直接用于[推理](./模型的推理Pipeline.ipynb)，一部分模型为了更好的定制性需要[训练](./模型的训练Train.ipynb)才能用在推理中，有关模型的下载和使用您可以查看[模型库的文档](../模型库/模型库介绍.ipynb)。

ModelScope的模型可以以非常简单的方式被调用加载：



In [1]:
from modelscope.models import Model
model = Model.from_pretrained('damo/nlp_structbert_word-segmentation_chinese-base')



该过程会在内部执行：
1. 使用字符串代表的model_id将模型下载到本地
2. 查看模型的[配置](../开发者使用指南/Configuration详解.ipynb)，找到模型名称和任务类型，并从模型注册工厂类中找到模型类
3. 将配置文件中model字段的参数传入给模型类并初始化该模型类
4. 载入二进制模型文件，把文件中的模型参数填充进模型类中

# 编写自定义模型

ModelScope提供了模型基类：



In [1]:
class Model(ABC):

    def __init__(self, model_dir, *args, **kwargs):
        self.model_dir = model_dir
        device_name = kwargs.get('device', 'gpu')
        verify_device(device_name)
        self._device_name = device_name

    def __call__(self, *args, **kwargs) -> Dict[str, Any]:
        return self.postprocess(self.forward(*args, **kwargs))

    @abstractmethod
    def forward(self, *args, **kwargs) -> Dict[str, Any]:
        pass

    def postprocess(self, inputs: Dict[str, Any], **kwargs) -> Dict[str, Any]:
        return inputs

    @classmethod
    def _instantiate(cls, **kwargs):
        return cls(**kwargs)

    @classmethod
    def from_pretrained(cls,
                        model_name_or_path: str,
                        revision: Optional[str] = DEFAULT_MODEL_REVISION,
                        cfg_dict: Config = None,
                        device: str = None,
                        *model_args,
                        **kwargs):
        pass

    def save_pretrained(self,
                        target_folder: Union[str, os.PathLike],
                        save_checkpoint_names: Union[str, List[str]] = None,
                        save_function: Callable = None,
                        config: Optional[dict] = None,
                        **kwargs):
        pass



以及PyTorch的模型基类：


In [1]:
class TorchModel(Model, torch.nn.Module):
    """ Base model interface for pytorch

    """

    def __init__(self, model_dir=None, *args, **kwargs):
        super().__init__(model_dir, *args, **kwargs)
        torch.nn.Module.__init__(self)

    def __call__(self, *args, **kwargs) -> Dict[str, Any]:
        pass

    def forward(self, *args, **kwargs) -> Dict[str, Any]:
        raise NotImplementedError
    
    ...



用户可以继承TorchModel或Model（非PyTorch场景）来编写新的模型类，下面的例子是一个自定义的PyTorch模型：



In [1]:
import torch.nn
from modelscope.models.base import TorchModel


class MyCustomModel(TorchModel):

    def __init__(self, model_dir, *args, **kwargs):
        super().__init__(model_dir, *args, **kwargs)
        self.linear = torch.nn.Linear(128, 1)

    def forward(self, input_tensor):
        return self.linear(input_tensor)

    @classmethod
    def _instantiate(cls, **kwargs):
        # 这个静态方法会在Registry中调用，可以在这里初始化模型
        return cls(**kwargs)



可以像这样调用它：


In [1]:
import torch
model = MyCustomModel(None)
input_tensor = torch.rand((1, 128))
print(model.forward(input_tensor))



可以将模型手动注册到模型工厂中以便在推理或训练中使用：



In [1]:
from modelscope.models.builder import MODELS
MODELS.register_module('my-custom-task', module_name='my-custom-model', module_cls=MyCustomModel)



这样，您可以通过from_pretrained方法调起：


In [1]:
from modelscope.utils.config import Config
from modelscope.models import Model
config = Config({'task': 'my-custom-task', 'model': {'type': 'my-custom-model'}})
config.dump('/tmp/configuration.json')
model = Model.from_pretrained('/tmp')
print(model.forward(input_tensor))


_instantiate方法用于通过静态方法拉起模型，这对于来自于transformers或fairseq等其他codebase的模型非常有用（这类模型通常通过静态方法初始化）。
在该方法存在于类中时，Registry会调用该方法拉起模型，在该方法不存在时，Registry会调用构造方法初始化模型。

Model类默认提供了一个_instantiate方法，该方法内部调用了模型的构造方法。


与之配套的，您可能需要自定义或复用预处理器部分，有关这里的介绍请参考[预处理器文档](./数据的预处理.ipynb)。

如果您自定义的模型是tensorflow或其他算法框架的，您需要继承Model类编写代码，ModelScope目前没有为tensorflow等框架提供特有基类。

有关ModelScope提供的模型的详细介绍，您可以参考API文档中的[模型文档]()。
